In [1]:
import os

In [2]:
%pwd

'c:\\Users\\unoto\\Documents\\my_project\\data_science\\Breast-Cancer-Prediction-MLOps\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\unoto\\Documents\\my_project\\data_science\\Breast-Cancer-Prediction-MLOps'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    scaler_path: Path
    penalty: str
    tol: float
    target_column: str

In [6]:
from breastCancer.constants import *
from breastCancer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.LogisticRegression
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            scaler_path=config.scaler_path,
            penalty=params.penalty,
            tol=params.tol,
            target_column=schema.name
        )

        return model_trainer_config

In [9]:
import pickle
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from breastCancer import logger

In [10]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        data = pd.read_csv(self.config.data_path)
        X = data.drop([self.config.target_column], axis=1)
        y = data[self.config.target_column]

        scaler = StandardScaler()
        X = scaler.fit_transform(X)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        model = LogisticRegression(penalty=self.config.penalty, tol=self.config.tol)
        model.fit(X_train, y_train)

        # save model and scaler
        with open(self.config.model_path, 'wb') as f:
            pickle.dump(model, f)
        with open(self.config.scaler_path, 'wb') as f:
            pickle.dump(scaler, f)
        
        logger.info("Model has been created.")

        

In [11]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2024-03-03 19:08:57,270: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-03 19:08:57,281: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-03 19:08:57,295: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-03 19:08:57,297: INFO: common: created directory at: artifacts]
[2024-03-03 19:08:57,300: INFO: common: created directory at: artifacts/model_trainer]
[2024-03-03 19:08:57,355: INFO: 1567981742: Model has been created.]
